# Store

> Module for constructing AppletStore.

In [ ]:
#| default_exp store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import yaml
import json
import os

from llmcam.oas_to_requests import *
from llmcam.fn_to_fc import *
from typing import Optional

In [ ]:
#| export
ToolBox = {}

## Dynamic API installation

In [ ]:
#| export
def load_oas(
    oas_url: str = "https://tie.digitraffic.fi/swagger/openapi.json",  # OpenAPI Specification URL
    destination: str = "api/road_digitraffic.json",  # Destination file
    overwrite: bool = False  # Overwrite existing file
) -> dict:  # OpenAPI Specification
    """Load OpenAPI Specification from URL or file."""
    # Create destination directory if it does not exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)

    # Download OpenAPI Specification if it does not exist or overwrite is True
    if not os.path.exists(destination) or overwrite:
        r = requests.get(oas_url)
        with open(destination, "w") as f:
            f.write(r.text)

    # Load OpenAPI Specification
    with open(destination, "r") as f:
        if destination.endswith(".json"):
            return json.load(f)
        elif destination.endswith(".yaml") or destination.endswith(".yml"):
            return yaml.load(f)
        else:
            raise ValueError("Invalid file format")

In [ ]:
show_doc(load_oas)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L23){target="_blank" style="float:right; font-size:smaller"}

### load_oas

>      load_oas (oas_url:str='https://tie.digitraffic.fi/swagger/openapi.json',
>                destination:str='api/road_digitraffic.json',
>                overwrite:bool=False)

*Load OpenAPI Specification from URL or file.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| oas_url | str | https://tie.digitraffic.fi/swagger/openapi.json | OpenAPI Specification URL |
| destination | str | api/road_digitraffic.json | Destination file |
| overwrite | bool | False | Overwrite existing file |
| **Returns** | **dict** |  | **OpenAPI Specification** |

Usage to download three DigiTraffic endpoints

In [ ]:
#| export
road_digitraffic = load_oas(
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    destination="api/road_digitraffic.json",
    overwrite=False
)
train_digitraffic = load_oas(
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    destination="api/train_digitraffic.json",
    overwrite=False
)
marine_digitraffic = load_oas(
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    destination="api/marine_digitraffic.json",
    overwrite=False
)

Test functions with GPT:

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get me information about the trains departing on 2017-11-09 with train number 1."),
])
complete(messages, toolbox_schema("https://rata.digitraffic.fi", train_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
Here is the information for Train Number 1 that departed on 2017-11-09:  - **Operator**: VR
(Operator UIC Code: 10) - **Train Type**: IC (InterCity) - **Train Category**: Long-distance -
**Commuter Line ID**: None - **Running Currently**: No - **Cancelled**: No - **Timetable Type**:
Regular - **Timetable Acceptance Date**: July 21, 2017  ### Timetable and Stops: 1. **Helsinki
(Station Code: HKI)**    - **Departure**: Scheduled at 05:28, Actual at 05:30:33 (3 mins delayed)
- **Track**: 7  2. **Pasila (Station Code: PSL)**    - **Arrival**: Scheduled at 05:33, Actual at
05:35:26 (2 mins delayed)    - **Departure**: Scheduled at 05:34, Actual at 05:37:13 (3 mins
delayed)    - **Track**: 4  3. **Tikkurila (Station Code: TKL)**    - **Arrival**: Scheduled at
05:43, Actual at 05:45:29 (2 mins delaye

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get me information about the trains departing on 2017-11-09 with train number 1.")
])
complete(messages, toolbox_schema("https://rata.digitraffic.fi", train_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
Here is the information about Train No. 1 that departed on November 9, 2017:  - **Operator**: VR -
**Train Type**: InterCity (IC) - **Train Category**: Long-distance - **Currently Running**: No -
**Cancelled**: No - **Timetable Type**: Regular  ### Timetable Details:  1. **Helsinki (HKI)**    -
Departure: Scheduled at 05:28, Actual: 05:30 (3 minutes late)    - Track: 7  2. **Pasila (PSL)**
- Arrival: Scheduled at 05:33, Actual: 05:35 (2 minutes late)    - Departure: Scheduled at 05:34,
Actual: 05:37 (3 minutes late)    - Track: 4  3. **Tikkurila (TKL)**    - Arrival: Scheduled at
05:43, Actual: 05:45 (2 minutes late)    - Departure: Scheduled at 05:44, Actual: 05:47 (3 minutes
late)    - Track: 3  4. **Lahti (LH)**    - Arrival: Scheduled at 06:16, Actual: 06:22 (7 minutes
late)    - Departure

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Are there any active nautical warnings?")
])
complete(messages, toolbox_schema("https://meri.digitraffic.fi", marine_digitraffic, fixup=generate_request))
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Are there any active nautical warnings?
>> Assistant:
Yes, there are currently several active nautical warnings. Here are a few of them:  1. **HESSUND
FAIRWAY:**    - **Warning Number:** 306    - **Type:** Navigational Warnings for Yachtsmen    -
**Location:** Archipelago Sea    - **Contents:** The fairway is closed for traffic from November 20,
2024, to April 30, 2025, due to bridge construction work.  2. **ARCHIPELAGO SEA, GULF OF FINLAND,
NORTHERN BALTIC:**    - **Warning Number:** 305    - **Type:** Coastal Warning    - **Contents:**
Military exercise "Freezing Winds" in progress from November 15 to November 29, 2024. Vessels are
advised to navigate with caution.  3. **BAY OF BOTHNIA - LOHTAJA:**    - **Warning Number:** 304
- **Type:** Coastal Warning    - **Contents:** Military firing exercises from November 13 to
November 16, 2024, in the sea area between Kokkola and Raahe. Traff

In [ ]:
#| export
def add_api_tools(
    service_name: str,  # Name of the API service
    base_url: str,  # Base URL of the API service
    oas_url: Optional[str] = None,  # OpenAPI Specification URL
    oas_destination: Optional[str] = None # OpenAPI Specification destination file
):
    """Add API tools to the toolbox."""
    # Load OpenAPI Specification
    if oas_url is None:
        oas_url = f"{base_url}/swagger/openapi.json"
    if oas_destination is None:
        oas_destination = f"api/{service_name}.json"
    oas = load_oas(oas_url, oas_destination, overwrite=True)

    # Create tool schema and append to toolbox
    global ToolBox
    ToolBox[service_name] = toolbox_schema(base_url, oas, fixup=generate_request)

In [ ]:
show_doc(add_api_tools)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L65){target="_blank" style="float:right; font-size:smaller"}

### add_api_tools

>      add_api_tools (service_name:str, base_url:str,
>                     oas_url:Optional[str]=None,
>                     oas_destination:Optional[str]=None, fixup:Callable=None)

*Add API tools to the toolbox.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| service_name | str |  | Name of the API service |
| base_url | str |  | Base URL of the API service |
| oas_url | Optional | None | OpenAPI Specification URL |
| oas_destination | Optional | None | OpenAPI Specification destination file |
| fixup | Callable | None | a fixup function to execute a REST API when a function name isn't found. |

## Dynamic functions from installed libraries

It is possible to dynamically import a function in installed library and execute it with `importlib`.

In [ ]:
#| export
from importlib import import_module
from typing import Callable, Any

In [ ]:
import json
from pathlib import Path

In [ ]:
func: Callable = getattr(import_module("llmcam.fn_to_fc"), "ask_gpt4v_about_image_file", None)
func

<function llmcam.fn_to_fc.ask_gpt4v_about_image_file(path: str) -> str>

In [ ]:
#| eval: false
def datadir(): return (Path('.') if os.path.isdir('data') else Path('..'))/'data' #FIXME: introduce LLMHOME and/or LLMDATA

In [ ]:
#| eval: false
json.loads(func(datadir()/"cap_2024.09.28_15:59:06_Presidentinlinna.jpg"))

{'timestamp': '2024-09-28T15:59:06',
 'location': 'Unknown',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 10,
  'building_height_range': '3-5 stories'},
 'vehicles': {'number_of_vehicles': 20,
  'number_of_available_parking_space': 5},
 'waterbodies': {'visible': True, 'type': 'harbor', 'number_of_boats': 12},
 'street_lights': {'number_of_street_lights': 10},
 'people': {'approximate_number': 30},
 'lighting': {'time_of_day': 'afternoon', 'artificial_lighting': 'minimal'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'daylight'}}

Auxiliary function to dynamically import and execute functions as defined in tools:

In [ ]:
#| export
def execute_function(
    function_name: str,  # Name of the function
    tools: list = [],  # The toolbox schema
    **kwargs  # Keyword arguments
) -> Any:  # Function output
    """Execute function with specified name."""
    # Get function from toolbox
    for tool in tools:
        if tool["function"]["name"] == function_name:
            module = tool["function"]["parameters"]["properties"]["module"]["default"]

    # Import module and get function
    if module is None:
        raise ValueError("Module not found")
    if module == "builtins":
        func: Callable = getattr(__builtins__, function_name, None)
    else:
        func: Callable = getattr(import_module(module), function_name, None)
    
    # Execute function
    if func is None:
        raise ValueError("Function not found")
    return func(**kwargs)

Usage with existing function:

In [ ]:
#| eval: false
json.loads(execute_function(
    "ask_gpt4v_about_image_file", 
    tools=[tool_schema(func)], 
    path=datadir()/"cap_2024.09.28_15:59:06_Presidentinlinna.jpg"
))

{'timestamp': '2024-09-28T15:59:06',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 10,
  'building_height_range': '3-5 stories'},
 'vehicles': {'number_of_vehicles': 20,
  'number_of_available_parking_space': 5},
 'waterbodies': {'visible': True, 'type': 'harbor', 'number_of_boats': 10},
 'street_lights': {'number_of_street_lights': 15},
 'people': {'approximate_number': 30},
 'lighting': {'time_of_day': 'afternoon', 'artificial_lighting': 'minimal'},
 'visibility': {'clear': True},
 'sky': {'visible': False, 'light_conditions': 'daytime'}}

Utility to add functions from installed libraries to ToolBox.

In [ ]:
#| export
def add_function_tools(
    service_name: str,  # Name of the service
    function_names: list[str],  # List of function names (with module prefix)
):
    """Add function tools to the toolbox."""
    # Initialize tools
    tools = []

    # Import functions
    for function_name in function_names:
        # Get module prefix
        module_prefix = function_name.split(".")
        if len(module_prefix) == 1:
            module_prefix = "builtins"
        else:
            module_prefix = ".".join(module_prefix[:-1])

        # Get function name without module prefix
        func_name = function_name.split(".")[-1]

        # Import function
        if module_prefix == "builtins":
            func: Callable = getattr(__builtins__, func_name, None)
        else:
            func: Callable = getattr(import_module(module_prefix), func_name, None)

        # Raise error if function not found
        if func is None:
            raise ValueError(f"Function not found: {function_name}")
        
        # Create tool schema
        tools.append(tool_schema(func))

    # Append tools to toolbox
    global ToolBox
    for function_name in function_names:
        ToolBox[service_name] = tools

In [ ]:
show_doc(add_function_tools)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L114){target="_blank" style="float:right; font-size:smaller"}

### add_function_tools

>      add_function_tools (service_name:str, function_names:list[str])

*Add function tools to the toolbox.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| service_name | str | Name of the service |
| function_names | list | List of function names (with module prefix) |

## ToolBox management

From the previous sections, we have implemented the utilities to add tools from API and functions. In this section, we can implement more utilities to manage the ToolBox, including:  

- Remove or Clean toolbox
- Save and load toolbox from `.json` file
- Select services and form usable tools 

In [ ]:
#| export
def remove_service(service_name: str):
    """Remove service from toolbox."""
    global ToolBox
    if service_name in ToolBox:
        del ToolBox[service_name]

#| export
def clean_toolbox():
    """Remove all services from toolbox."""
    global ToolBox
    ToolBox = {}

In [ ]:
#| export
def save_toolbox(destination: str = "toolbox.json"):
    """Save toolbox to file."""
    with open(destination, "w") as f:
        json.dump(ToolBox, f)
    
#| export
def load_toolbox(destination: str = "toolbox.json"):
    """Load toolbox from file."""
    global ToolBox
    with open(destination, "r") as f:
        ToolBox = json.load(f)

In [ ]:
#| export
def extract_tools_from_services(
    services: list[str] = []  # List of service names
) -> list:  # List of tools
    """Extract tools from services."""
    # Initialize tools
    tools = []

    # Extract tools from services
    global ToolBox
    for service in services:
        if service in ToolBox:
            tools.extend(ToolBox[service])

    # Raise error if no tools found
    if len(tools) == 0:
        raise ValueError("No tools found")
    
    # Raise error if too many tools
    if len(tools) > 128:
        raise ValueError("Too many tools for using GPT-4. Maximum number of tools is 128.")

    # Return tools
    return tools

In [ ]:
show_doc(extract_tools_from_services)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L179){target="_blank" style="float:right; font-size:smaller"}

### extract_tools_from_services

>      extract_tools_from_services (services:list[str]=[])

*Extract tools from services.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| services | list | [] | List of service names |
| **Returns** | **list** |  | **List of tools** |

For our demo, we can add some default services from DigiTraffic and our existing `llmcam` module:

In [ ]:
#| export
add_api_tools(
    service_name="road_digitraffic",
    base_url="https://tie.digitraffic.fi",
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    oas_destination="api/road_digitraffic.json"
)

In [ ]:
#| export
add_api_tools(
    service_name="train_digitraffic",
    base_url="https://rata.digitraffic.fi",
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    oas_destination="api/train_digitraffic.json"
)

In [ ]:
add_api_tools(
    service_name="marine_digitraffic",
    base_url="https://meri.digitraffic.fi",
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    oas_destination="api/marine_digitraffic.json"
)

In [ ]:
#| export
add_function_tools(
    service_name="ytube_live",
    function_names=[
        "llmcam.fn_to_fc.capture_youtube_live_frame_and_save",
        "llmcam.fn_to_fc.ask_gpt4v_about_image_file"
    ]
)

However, if we have both tools stemming from functions and API requests, we can benefit from an auxiliary function to redirect the calls into sub-auxiliary functions:

In [ ]:
#| export
def redirect_tool_calls(
    function_name: str,  # Name of the function
    tools: list = [],  # List of tools
    **kwargs  # Keyword arguments
):
    """Redirect tool calls to the appropriate function."""
    for tool in tools:
        if tool["function"]["name"] == function_name:
            if "module" in tool["function"]["parameters"]["properties"]:
                return execute_function(
                    function_name=function_name, 
                    tools=tools, 
                    **kwargs)
            
            elif "url" in tool["function"]["parameters"]["properties"]:
                return generate_request(
                    function_name=function_name, 
                    tools=tools, 
                    **kwargs)
    
    raise ValueError("Function not found")

In [ ]:
show_doc(redirect_tool_calls)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L231){target="_blank" style="float:right; font-size:smaller"}

### redirect_tool_calls

>      redirect_tool_calls (function_name:str, tools:list=[], **kwargs)

*Redirect tool calls to the appropriate function.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| function_name | str |  | Name of the function |
| tools | list | [] | List of tools |
| kwargs |  |  |  |

Test integrating with a GPT chatbot that has both Road API from DigiTraffic and Youtube Live functions.

In [ ]:
tools = extract_tools_from_services(["ytube_live", "road_digitraffic"])

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get the weather camera information for the stations with ID C01503 and C01504."),
])
complete(messages, tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with IDs **C01503** and **C01504**:  ###
Station C01503 (Road 51 Inkoo)  - **Name:** Tie 51 Inkoo - **Camera Type:** BOSCH - **Coordinates:**
[23.99616, 60.05374] - **Nearest Weather Station ID:** 1013 - **Collection Status:** GATHERING -
**Collection Interval:** 600 seconds - **Location:** Inkoo, Uusimaa - **Start Time:** 2001-08-01
**Presets:** 1. **Inkooseen**    - **Resolution:** 1280x720    - **Direction:** INCREASING_DIRECTION
- ![Image](https://weathercam.digitraffic.fi/C0150301.jpg) 2. **Hankoon**    - **Resolution:**
1280x720    - **Direction:** DECREASING_DIRECTION    -
![Image](https://weathercam.digitraffic.fi/C0150302.jpg) 3. **Tienpinta**    - **Resolution:**
1280x720    - **Direction:** SPECIAL_DIRECTION    -
![Image](https:/

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Can you capture a YouTube Live and extract information from it? You can use the default link."),
])
complete(messages, tools)
print_msgs(messages)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading web creator player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
PRPC eRe ECM ERE ot Elm
cap_2024.11.16_13:35:35_unclear.jpg
>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Can you capture a YouTube Live and extract information from it? You can use the default link.
>> Assistant:
I've captured a frame from the YouTube Live video and extracted the following details:  -
**Timestamp:** 16th November 2024, 13:28 - **Location:** Valkosaari - **Dimensions:** 1280 x 720
### Scene Analysis: - **Buildings:**    - Number: 3   - Height Range: 2-3 stories - **Vehicles:**
- Number: 0 - **Parking Space:**    - Available: 0 - **Water Bodies:**    - Visible: Yes   - Type:
Harbor   - Number of Boats: 2 - **Street Lights:**    - Number: 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()